## Spatial Modeling and Analytics Try-it Notebook #2
### Calculating distances and areas

## Reminder
<a href="#/slide-2-0" class="navigate-right" style="background-color:blue;color:white;padding:8px;margin:2px;font-weight:bold;">Continue with the lesson</a>

<br>
</br>
<font size="+1">

By continuing with this lesson you are granting your permission to take part in this research study for the Hour of Cyberinfrastructure: Developing Cyber Literacy for GIScience project. In this study, you will be learning about cyberinfrastructure and related concepts using a web-based platform that will take approximately one hour per lesson. Participation in this study is voluntary.

Participants in this research must be 18 years or older. If you are under the age of 18 then please exit this webpage or navigate to another website such as the Hour of Code at https://hourofcode.com, which is designed for K-12 students.

If you are not interested in participating please exit the browser or navigate to this website: http://www.umn.edu. Your participation is voluntary and you are free to stop the lesson at any time.

For the full description please navigate to this website: <a href="../../gateway-lesson/gateway/gateway-1.ipynb">Gateway Lesson Research Study Permission</a>.

</font>

In this try-it you'll download some data and calculate distances between points. 

Once again we'll work in a notebook without using slides. Scroll down through the series of code blocks, executing them as you go. Run through this notebook as presented without making any changes. Then when you're done, try experimenting with the code by making minor modifications. Enjoy!

Remember for each of the code chunks below, click the arrow to the left of the box. Be patient, sometimes these take a few seconds to execute. Wait for the asterisk to change into a number.

# Key modules and libraries we'll use

There are three Python packages we import a lot in spatial analytics and modeling: pandas, geopandas and shapely. 
- <a href="https://pandas.pydata.org/docs/">Pandas</a> is "a library providing high-performance, easy-to-use data structures and data analysis tools." You will use it in many of your python operations, not just geospatial ones. Pandas' key data structure is the dataframe (often labelled a "df"), a simple tabular structure like a spreadsheet. 
- <a href="https://geopandas.org/">Geopandas</a> is built on pandas to "extend the datatypes used by pandas to allow spatial operations on geometric types." Geopandas' key data structure is the geodataframe (often labelled "gdf"). 
- Finally, <a hred="https://shapely.readthedocs.io/en/stable/#">shapely</a> handles "manipulation and analysis of planar geometric objects."

In the code below, we execute "import pandas as pd". Python programmers like to use shortcuts so setting "pd" to refer to pandas is pretty standard. Likewise, you'll see below we'll import geopandas as "gpd". Get used to it! 

Other libraries and modules we use here are:
- <a href="https://docs.python.org/3/library/csv.html">The CSV module</a> implements classes to read and write tabular data in CSV format.
- <a href="https://docs.python-requests.org/en/master/">Requests</a> provides a simple way to send http requests to APIs from which you wish to pull data.
- <a href="https://python-visualization.github.io/folium/quickstart.html">Folium</a> is one of the packages you can use to make "beautiful, interactive maps with Python and Leaflet.js."

In [2]:
import pandas as pd
import geopandas as gpd
import shapely
import csv
import requests
import warnings
warnings.filterwarnings('ignore') # Hide warnings

# Get the Data

There are tons of tabular data with lat/long attributes on the web, much of it provided in CSV (comma separated values) format, which is a simple text file with the attribute data in each row separated by commas. It's super simple to grab a CSV file and turn it into something you can do spatial analysis on in a notebook and Python. 

For this exercise we're going to look at abandoned wells in the State of New York and we've found a source of tabular data at the general website <a href="https://www.data.gov/">data.gov</a>, "The home of the U.S. Government’s open data". 

Our data for this exercise can be found at https://catalog.data.gov/dataset/abandoned-wells. It is a list of wells that are regulated under the Oil, Gas and Solution Mining Law (ECL Article 23) in New York State that are abandoned and not plugged.

Let's get the data and look at it. 

In [3]:
path = 'https://data.ny.gov/api/views/vgue-bamz/rows.csv'
wells = pd.read_csv(path)

wells.info()
wells.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6851 entries, 0 to 6850
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   COMPANY NAME       6851 non-null   object 
 1   COUNTY             6851 non-null   object 
 2   TOWN               6851 non-null   object 
 3   API WELL NUMBER    6851 non-null   int64  
 4   WELL NAME          6851 non-null   object 
 5   WELL STATUS CODE   6851 non-null   object 
 6   WELL TYPE CODE     6851 non-null   object 
 7   SURFACE LONGITUDE  6682 non-null   float64
 8   SURFACE LATITUDE   6682 non-null   float64
 9   VERIFIED LOCATION  6851 non-null   object 
 10  REGION             6851 non-null   int64  
 11  Georeference       6682 non-null   object 
dtypes: float64(2), int64(2), object(8)
memory usage: 642.4+ KB


,COMPANY NAME,COUNTY,TOWN,API WELL NUMBER,WELL NAME,WELL STATUS CODE,WELL TYPE CODE,SURFACE LONGITUDE,SURFACE LATITUDE,VERIFIED LOCATION,REGION,Georeference
0,"Duchscherer, William J.",Orleans,Shelby,31073000000000,Cook D R 1,UM,DW,-78.44156,43.185860,NO,8,POINT (-78.44156 43.18586)
1,Unknown,Erie,Elma,31029000000000,Cotton G G 1,UN,GD,-78.60549,42.863370,NO,9,POINT (-78.60549 42.86337)
2,Alden Batavia Natural Gas Co.,Erie,Alden,31029000000000,Nobel Nuwer,UN,GD,-78.56141,42.915080,NO,9,POINT (-78.56141 42.91508)
3,Valley Drilling Corp. of America,Chautauqua,Busti,31013200000000,D Eckstrom 5,UL,OD,-79.29716,42.006463,YES,9,POINT (-79.29716 42.006463)
4,"Lee Oil Company, Inc.",Steuben,West Union,31101600000000,Young P-44,UL,IW,-77.71145,42.093610,NO,8,POINT (-77.71145 42.09361)


# Clean and prepare the data for analysis

By looking at this info, we can see it has some problems (as raw data usually does). There are 6851 rows (entries), but only 6682 have latitude and longitude values. So, we can delete those as they will be of no use to us. 

Also, you'll see there is a column called Georeference that appears to be a merge of the lat and long values. As you'll discover when working with Python, and many other languages like R, different modules have different data format requirements. We need a geometry column for the coding we're going to do here and it's not clear if this merged column suits the format requirements of the modules we'll use, so just to avoid confusion, we'll delete that column and generate a new geometry one within geopandas.  

In [4]:
#first we delete the column "Georeference"
del wells['Georeference']

#then we remove all rows that have no lat and long data
wells = wells[wells['SURFACE LONGITUDE'].notna() & wells['SURFACE LATITUDE'].notna()]

wells.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6682 entries, 0 to 6850
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   COMPANY NAME       6682 non-null   object 
 1   COUNTY             6682 non-null   object 
 2   TOWN               6682 non-null   object 
 3   API WELL NUMBER    6682 non-null   int64  
 4   WELL NAME          6682 non-null   object 
 5   WELL STATUS CODE   6682 non-null   object 
 6   WELL TYPE CODE     6682 non-null   object 
 7   SURFACE LONGITUDE  6682 non-null   float64
 8   SURFACE LATITUDE   6682 non-null   float64
 9   VERIFIED LOCATION  6682 non-null   object 
 10  REGION             6682 non-null   int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 626.4+ KB


OK, that got rid of the rows with no location data. But this is still a lot of data for a little exercise, so we'll extract only the wells in Region 8 and we might as well trim off the ones for which location is not verified. 

In [5]:
wells8 = wells[(wells['VERIFIED LOCATION']=='YES') & (wells['REGION']==8)]

wells8.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 239 to 6711
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   COMPANY NAME       77 non-null     object 
 1   COUNTY             77 non-null     object 
 2   TOWN               77 non-null     object 
 3   API WELL NUMBER    77 non-null     int64  
 4   WELL NAME          77 non-null     object 
 5   WELL STATUS CODE   77 non-null     object 
 6   WELL TYPE CODE     77 non-null     object 
 7   SURFACE LONGITUDE  77 non-null     float64
 8   SURFACE LATITUDE   77 non-null     float64
 9   VERIFIED LOCATION  77 non-null     object 
 10  REGION             77 non-null     int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 7.2+ KB


So now we've got how many entries left?

[77] 

OK, that's plenty small now. 

Now we need to do a bit more data munging so we can start doing geometry calculations. Here we use shapely to create a new column at the far right called geometry which contains point features composed of the lat and long values. 

In [6]:
from shapely.geometry import Point

# Make a list that contains a series of point geometries from each pair of lat/long values.
geometry_latlon = [Point(xy) for xy in zip(wells8['SURFACE LONGITUDE'], wells8['SURFACE LATITUDE'])]

# Append the list to the wells8 dataframe to create a new column "geometry"
wells8['geometry'] = geometry_latlon 

wells8.head()

,COMPANY NAME,COUNTY,TOWN,API WELL NUMBER,WELL NAME,WELL STATUS CODE,WELL TYPE CODE,SURFACE LONGITUDE,SURFACE LATITUDE,VERIFIED LOCATION,REGION,geometry
239,Landie Co.,Steuben,Addison,31101000000000,Ackerson 1,UL,DW,-77.21019,42.098220,YES,8,POINT (-77.21019 42.09822)
262,"Chris Glade Oil & Gas, LLC",Schuyler,Reading,31097200000000,Fierro 1,UL,GW,-76.90737,42.444854,YES,8,POINT (-76.90737 42.444854)
274,Empire Gas & Fuel,Steuben,Howard,31101000000000,Willis 1,UL,GD,-77.52383,42.363910,YES,8,POINT (-77.52383 42.36391)
600,"Republic Light, Heat & Power",Genesee,Pavilion,31037000000000,Fagen,UL,DH,-78.03502,42.907010,YES,8,POINT (-78.03502 42.90701)
680,Belmont Quadrangle Drilling,Chemung,Erin,31015000000000,Treat 1,UL,DW,-76.70758,42.237270,YES,8,POINT (-76.70757999999999 42.23727)


And finally, let's convert the pandas dataframe into a geopandas geodataframe and specify that the coordinate reference system (CRS) for the geometry is WGS84 (lat/long). This is EPSG code 4326.

[The <a href='http://www.epsg-registry.org/'>EPSG Geodetic Parameter Dataset</a> is a structured dataset of CRS and Coordinate Transformations. It was originally compiled by the, now defunct, European Petroleum Survey Group, hence the acronymn, though it is no longer maintained by that group.]

In [7]:
wells_gdf = gpd.GeoDataFrame(wells8, geometry = 'geometry')
wells_gdf.crs = 4326

Now that our geodataframe has a CRS, we can convert the lat/long coordinates to the UTM coordinate system simply by providing the EPSG code for the UTM zone covering this location (18N). That's EPSG code 32618. That will make our data finally ready for some Cartesian geometry calculations! (Remember: Data prep often takes a large portion of most spatial analytics project time.)

In [8]:
wells_gdf_utm = wells_gdf.to_crs(epsg='32618')
wells_gdf_utm.head()

,COMPANY NAME,COUNTY,TOWN,API WELL NUMBER,WELL NAME,WELL STATUS CODE,WELL TYPE CODE,SURFACE LONGITUDE,SURFACE LATITUDE,VERIFIED LOCATION,REGION,geometry
239,Landie Co.,Steuben,Addison,31101000000000,Ackerson 1,UL,DW,-77.21019,42.098220,YES,8,POINT (317234.409 4663045.432)
262,"Chris Glade Oil & Gas, LLC",Schuyler,Reading,31097200000000,Fierro 1,UL,GW,-76.90737,42.444854,YES,8,POINT (343138.508 4700932.198)
274,Empire Gas & Fuel,Steuben,Howard,31101000000000,Willis 1,UL,GD,-77.52383,42.363910,YES,8,POINT (292171.249 4693267.453)
600,"Republic Light, Heat & Power",Genesee,Pavilion,31037000000000,Fagen,UL,DH,-78.03502,42.907010,YES,8,POINT (252239.163 4754958.213)
680,Belmont Quadrangle Drilling,Chemung,Erin,31015000000000,Treat 1,UL,DW,-76.70758,42.237270,YES,8,POINT (359106.805 4677532.105)


You'll see the content of these two geodatabases is identical except for the final geometry column that displays different coordinate units. 

And a good final step in our data preparation stage is to take a look at the final data on a map. 

In [9]:
import folium

mymap = folium.Map(location = [42.58758, -77.16301], tiles='OpenStreetMap' , zoom_start = 8) 

for _, r in wells_gdf.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']) 
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, 
                style_function = lambda x: {'color': 'red', 'weight': 1,  'fillColor': 'YlGnBu'})
    folium.Popup(f"<i>Well Name: {r['WELL NAME']}, <br> Well Type Code: {r['WELL TYPE CODE']}, <br> Company Name: {r['COMPANY NAME']}</i>", min_width=200, max_width=400).add_to(geo_j)
    folium.Tooltip(f"<i>Well Name: {r['WELL NAME']}, <br> Well Type Code: {r['WELL TYPE CODE']}, <br> Company Name: {r['COMPANY NAME']}</i>").add_to(geo_j)
    geo_j.add_to(mymap)

mymap

# Find distances

FINALLY, let's do some spatial analytics! Earlier we looked at the algorithm for calculating the distance between two points. Good news! That algorithm is built into geopandas, so all we have to do is invoke the "distance" function. 

One more nomenclature thing - how to pull out a single cell in our geodataframe? Like this:

     gdf_name.column_name.iloc[row_number]. So the UTM geometry of the two wells are:

In [10]:
print(wells_gdf_utm.geometry.iloc[0])
print(wells_gdf_utm.geometry.iloc[1])

POINT (317234.4093253907 4663045.432101264)
POINT (343138.5076766623 4700932.197860493)


So now we can calculate the distance in meters between the first well and all others. We start by defining an empty list variable and then iterate through the dataframe, one row at a time, adding each calculated distance to the list.

In [11]:
distances = []

for i in range(len(wells_gdf_utm)):
    distances.append(wells_gdf_utm.geometry.iloc[0].distance(wells_gdf_utm.geometry.iloc[i]))

distances   

[0.0,
 45895.853092486344,
 39262.355173427626,
 112571.49473623892,
 44307.575164536174,
 51250.19566193865,
 14770.609241002017,
 14427.496801142635,
 15169.478563503353,
 44695.08625257195,
 54496.89588953737,
 71951.44025976695,
 73198.53812317128,
 43754.48605156225,
 19507.179329905575,
 46509.85474062015,
 43618.150772199355,
 28894.925643442646,
 77760.82667417619,
 36787.19863226996,
 15259.853832269764,
 43369.57343899096,
 93045.11178941617,
 14014.833179234296,
 85230.97388910648,
 32921.070680161676,
 37768.115317076605,
 4514.934042411043,
 34084.795895643336,
 136642.46618078402,
 93304.20376482642,
 128916.75103039067,
 107876.38667296417,
 70753.29446952631,
 30958.056242221977,
 43949.67951221509,
 15182.157693187228,
 114459.20755756949,
 73210.16263067955,
 99437.90479475886,
 87120.49276126162,
 32763.101482516246,
 46761.07934862582,
 46205.864598809734,
 27287.700380793627,
 21010.40804950479,
 98730.84852232986,
 85998.05691813458,
 95231.08486691453,
 5850.9054

And let's do the big one! Build a matrix that shows the distance between every point and every other point!

In [12]:
# generate column names e.g., Point 0, Point 1, etc
pdf = {}
col_names = [f'Point {i}' for i in range(len(wells_gdf_utm))]
pdf.update({' ': col_names}) 

distances = []
for i in range(len(wells_gdf_utm)):
    distances = []
    for j in range(len(wells_gdf_utm)):
        distances.append(wells_gdf_utm.geometry.iloc[i].distance(wells_gdf_utm.geometry.iloc[j]))
        pdf.update({f'Point {i}': distances})

dist_matrix = pd.DataFrame(pdf)
dist_matrix.set_index(' ', inplace=True) # Y axis naming style
dist_matrix

,Point 0,Point 1,Point 2,Point 3,Point 4,Point 5,Point 6,Point 7,Point 8,Point 9,...,Point 67,Point 68,Point 69,Point 70,Point 71,Point 72,Point 73,Point 74,Point 75,Point 76
,,,,,,,,,,,,,,,,,,,,,
Point 0,0.000000,45895.853092,39262.355173,112571.494736,44307.575165,51250.195662,14770.609241,14427.496801,15169.478564,44695.086253,...,20477.472971,81213.894602,9011.999093,43763.625998,88213.851209,46392.190372,12388.143867,30129.254746,97397.489835,34307.150372
Point 1,45895.853092,0.000000,51540.370286,105742.617922,28329.328491,38431.129188,49391.633581,49269.346804,45417.835028,17170.534389,...,55452.715529,78711.820027,54504.170858,18167.102344,67411.275269,38710.616432,56287.154575,62058.473678,53250.769644,64832.685914
Point 2,39262.355173,51540.370286,0.000000,73486.879496,68760.234200,78704.075291,25993.698035,26287.371206,24173.447226,62415.375818,...,24732.005446,42107.814426,39846.130140,35848.483974,55335.696593,75572.261987,36690.159219,21872.701740,86519.970293,21400.514969
Point 3,112571.494736,105742.617922,73486.879496,0.000000,131967.780751,142568.754627,99410.238750,99714.133438,97619.267516,122149.579891,...,97094.406446,31380.939462,113176.814132,88278.818093,43622.501097,141152.351456,109674.521821,90741.653168,102667.855827,87805.041627
Point 4,44307.575165,28329.328491,68760.234200,131967.780751,0.000000,10602.227265,55201.621430,54932.134543,52592.412440,11434.859789,...,61983.998215,103207.423776,52961.993955,43890.340129,95386.804011,10506.874131,56695.447578,70962.242502,75457.108348,74734.096652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Point 72,46392.190372,38710.616432,75572.261987,141152.351456,10506.874131,5652.645450,59038.750531,58736.857979,57042.726040,21941.429738,...,65689.509737,111847.193258,54330.197006,53554.823694,105430.159122,0.000000,58543.898867,75082.421289,85365.095018,79057.924219
Point 73,12388.143867,56287.154575,36690.159219,109674.521821,56695.447578,63509.404401,11291.220830,11091.395858,14863.284760,56743.603216,...,12746.421976,78419.356462,4747.603371,51063.419565,89997.267981,58543.898867,0.000000,21277.766678,105997.522677,25358.686602
Point 74,30129.254746,62058.473678,21872.701740,90741.653168,70962.242502,79408.755075,16057.751788,16367.891103,18386.579703,67937.791816,...,9655.616492,60073.080922,25817.452226,50649.940301,77102.918364,75082.421289,21277.766678,0.000000,104915.704477,4193.002784


Very cool! Now there's plenty of statistics we could do on this matrix (average, max, min, etc.) and we could do more advanced matrix math to calculate clusters and spatial autocorrelation! But that's enough for now.

<font size="+1"><a style="background-color:blue;color:white;font-weight:bold;" 
href="sma-5.ipynb">OK, let's go back to the final part of the lesson!</a></font>